# Cubesat Camera Field of View

Find the plate scale and field of view of a small camera on a cubesat in [Low Earth Orbit](http://www.universetoday.com/85322/low-earth-orbit/).

In [ ]:
print """It's a camera"""